In [14]:
from transformers import pipeline
summarizer = pipeline("summarization")
ARTICLE = """ A Louisiana sheriff's deputy died by suicide on Monday after recording and posting to social media videos of himself condemning institutionalized racism and describing his struggle as a Black law enforcement officer in a system that he says condones police brutality against Black people.

Lafayette Parish Deputy Clyde Kerr III, 43, died from a self-inflicted gunshot wound to the head, according to a preliminary coroner's report. He was outside the Lafayette Parish Sheriff's Office, according to Capt. John Mowell.
Kerr became a deputy in 2015, Mowell said.
He was a New Orleans native, Army veteran and father of two boys.
In the videos posted online, he spoke about the police killings of Black Americans such as Breonna Taylor in Louisville, Kentucky, and George Floyd in Minneapolis -- which sparked nationwide protests last summer against police brutality and racial injustice.
"I've had enough of all of this nonsense, serving a system that does not give a damn about me or people like me," Kerr said in one video, speaking directly to the camera. "You have no idea how hard it is to put a uniform on in this day and age with everything that's going on."
CNN has confirmed Kerr's identity in the videos with Michael Robinson, who described Kerr as one of his closest friends. Kerr filmed two videos on January 29 and a final one on January 31. All three were posted to his YouTube account just hours before his death, according to Robinson.
In his last video, Kerr described himself as a "stellar" deputy who dedicated his life to the service of others. He also called for additional mental health resources for police officers, including more frequent psychiatric evaluations.
"This is my protest against police brutality and everything else that comes along with it in this broken, wicked, worldly system that does not give a damn about people," the deputy said.
Kerr, who served in Afghanistan and Iraq as a soldier, was a school resource officer at St. Genevieve Elementary School and St. Genevieve Middle School in Lafayette.
Robinson said he met Kerr in 1995 as they were moving into their dorms at the University of Louisiana at Lafayette, where the deputy studied criminal justice.
"He just loved being an officer," Robinson said. "He just didn't like some of the things behind the scenes, such as practices that needed to change for the better. He had great ideas about what could be done to make it better and heal the relationship with the public."
Robinson told CNN that the death of Floyd last May weighed heavily on Kerr. He confided in Robinson last summer about a difficult conversation he had with his teenage son, who watched the widely circulated video of Floyd's killing.
"He told me that he thought about resigning just because of how everything was, and I told him if he did that in this climate, I don't think anybody could fault him for it," Robinson said. "But I said, 'Man, those kids would really miss you.' That's basically the reason he was still there."
In a statement to CNN, the Lafayette Parish Sheriff's Office said they are "heartbroken" from the loss of Kerr, who "took his own life earlier this week and left behind so many friends and coworkers who cared for him deeply. Our thoughts and prayers are with Deputy Kerr's family, as we all struggle to process this together."
"""

In [15]:
print(summarizer(ARTICLE, max_length=200, min_length=30, do_sample=False))

[{'summary_text': " Lafayette Parish Deputy Clyde Kerr III, 43, died from a self-inflicted gunshot wound to the head, coroner's report says . He was a New Orleans native, Army veteran and father of two boys . In the videos, he spoke about the police killings of Black Americans such as Breonna Taylor and George Floyd ."}]


In [1]:
import torch

In [2]:
import torchvision

# Transformer reintenting the wheel

In [4]:
import torch
from torch import nn

In [5]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [7]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
        return x

In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [12]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)
   
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

In [13]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [14]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [15]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda()
def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [16]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [17]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output
# we don't perform softmax on the output as this will be handled 
# automatically by our loss function

In [18]:
d_model = 512
heads = 8
N = 6
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(FR_TEXT.vocab)
model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

NameError: name 'EN_TEXT' is not defined

In [28]:
from transformers.models.bert import modeling_bert

from transformers import BertModel, BertConfig
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()
# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)
# Accessing the model configuration
configuration = model.config

In [29]:
modeling_bert.BertAttention(configuration)

BertAttention(
  (self): BertSelfAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (output): BertSelfOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
text = '''
    Hey there, Reddit… I’m writing this here because I don’t know where else to vent my frustration and let it all out. My name is Charlotte, I’m fifteen years old and I think I am in trouble. I am hiding in the basement while typing this, hoping to escape the fury of the monster upstairs.

I’m really scared and my daddy doesn’t believe me. He never believes me when I tell him that what he did is wrong. He has changed a lot in this past year and he had help with that.

He doesn’t take advice from anyone anymore and that woman enables all his erratic behavior and they drink that thing and start laughing and making stupid noises or animal sounds.

I hate her. I really do.

Since she came into our lives everything changed for the worse. My mommy had a nervous breakdown and she had to go for a while into one of those centers for people who lose control.

It’s not so funny that people call them funny farms. There’s nothing funny about them, you know?

Since they divorced, daddy changed a whole lot. Especially since this woman came into our lives and I am forced to live with her every single day until mommy gets out of the institution.

Daddy doesn’t see her for what she is. A destroyer of homes, a wrecker of families… She is the cause my parents divorced, that much I know.

I may not be the sharpest tool in the shed, but I have a gift. I see people for what they really are. I mean, for what they really are. Sometimes they are just normal people, friendly people. Other times, they are beautiful on the outside but they hide their true ugly self on the inside.

Some of them though are not people. They are monsters hiding in a skin suit. Mimicking a normal person. Taking over their host and manipulating others to do their bidding.

Such is the case with this woman my daddy brought home. She is evil and she scares me a lot. Daddy says that I’m like that because I’m just in that period of my life where I am angry all the time.

He tells me all teens are like that. Angry at their parents, teachers, and society in general. I’m not like that, I swear. I just hate that woman who took my daddy away from me and my mommy.

That vile, twisted and wretched monster. I’ll make her go away tonight. Yes, I will.

Daddy says that he and mommy didn’t get the divorce because of the woman, but because they didn’t get along anymore. The flame was gone, he says.

That’s a lie, I know it for sure. They know each other from work and that’s how it all started. I’m not stupid, I can see and sense things.

I remember that when I first saw her true self, I gasped for air and froze, standing still like a statue. Trembling with fear I saw her disgusting face and it scared me a lot. She gently touched my face with her finger and asked me if everything was alright to which I nodded. She grinned and ruffled my hair like I was some sort of pet.

Sorry for the bad language, but I fucking hate that bitch.

The reason I am hiding here is that I dropped a plate and it smashed into a million bits on the kitchen floor. I was trying to help the monster do the dishes. But at the moment the plate broke, she jumped and she started yelling at me, calling me all kinds of names, bad names that I don’t to repeat here.

She lost control for a moment and that’s when I saw her true face again. She’s just an evil old witch who feeds and prey on weak people like my dad. She pulled a knife and pointed it at me. Then she said that I shouldn’t mention the incident to daddy because it will only get worse and on top of that he won’t believe it, so it was pointless.

You see, I didn’t tell you yet but daddy has some sort of purple cloud hanging above his head whenever they are together. I think he’s been hexed and that’s why he can’t think straight anymore, but I know that if I can make the woman go away or kill her, I’ll have my daddy back with my mommy in no time.

She put a love curse on him and now he can only think about her and not anyone else.

The following part is very hard to write, but I’ll give it a try. After the knife incident, I tried running away but she kept me in place.

Then the kitchen changed to something else. A different world filled with blackness, where the winds were howling in a million different tones and where I heard the cries and screams of people trapped in there.

I started crying and yelling and I called for my mommy, my voice echoing through the darkness. Then I heard her say that no one is going to save me and I saw my mommy in chains at that institution, it was horrible and I was beyond scared out of my mind.

Then I heard beasts howling in the distance and I thought they were coming to get me and I screamed and screamed and I couldn’t get out of that evil place.

The monster witch woman grabbed my arms, bruising them in the process. “This is the place for naughty children like you! If you misbehave again, this is where you’ll end up!”

The beasts were coming ever closer and her grip on my arm was tightening that I thought she would break it. That’s when I screamed again and white light came out of my mouth and eyes, blasting the witch away, hurting her I guess and then I woke up here in the basement.

It’s like someone or something took care of me and instructed me with a plan.

There is a small gas canister here beside because sometimes daddy works here for his car things.

I’m a smoker. I know it’s not healthy but some teens do it right? Like my daddy said, some of us pretend to be misfits and pick up bad habits. I think this might help me in my current situation because I have a lighter.

I will burn the witch while she sleeps, right after daddy goes to work. He starts the night shift in a few hours and I’ll just stay here until I hear him leave. I’m just hoping my phone doesn’t die.

A few hours pass and I look outside the small window from my basement and see it’s already night. I check with my phone to make sure that I’m not being deceived by some external forces and it says 10 PM.

I hear daddy telling the witch goodbye and making kissing sounds. Gross, yuck.

He then leaves for work, the front door of the house closing shut behind him. My heart starts beating like crazy inside my chest and sweat is coming down my temples. I’ve never been more afraid than right now in my entire life.

The bitch witch is sleeping; I can hear her snoring echoing throughout the house.

Good for me.

The gas canister is very small and that makes it easier for me to carry it. I make sure to check the lighter still has gas of its own and it does.

Phew, all good so far.

I slowly open the door to the basement and remove my shoes so I don’t make any unwanted noises. I go upstairs and slowly listen to the witch’s snoring. She seems to talk gibberish in her sleep. Holy hell, I am scared and my hands are sweaty and shaking.

I slowly turn the doorknob, making sure I don’t make a sound. I enter the room and see her. She sleeps in her true form, this evil witch. She’s a monster that needs to be killed. Right here and now.

I douse her in gasoline and she jumps out of bed, screaming and kicking demanding to know what the hell is going on. The bedsheet sticks on her skin, and she can’t seem to free her hands.

“What are you doing, you crazy child?” she screams, her eyes bulging in disbelief.

“I’m just lighting a cigarette. You evil fucking monster,” I tell her, grinning.

I struck the lighter and throw it on the witch. The flames engulf her body and she’s screaming and kicking and I’m scared shitless as she tries to come and catch me but I run outside of the room.

Her dying screams are evaporating in the stillness of the night. I’m crying tears of joy, still not knowing how I managed to pull that off. She’s giving her final breath and I go back to see just a pile of grey and black ash.

I put it in a metal box and bury it in my garden outside. I don’t know how I didn’t burn down the house. The only things that burned were the bedsheet and the witch.

“You’re one of us now. The ones who serve the light. You are so brave and strong,” I hear a voice telling me.

I just shrug it off, saying OK and thanking the voice for their help. The bruise from earlier still hurts, but it will heal soon.

Now I just wait for my daddy to come home and I can’t wait for mommy to get out of the hospital.

I will never forget this experience and I will do anything necessary to protect my family.

    '''

In [6]:
doc = nlp(text)
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [24]:
emb = sbert.encode([str(sent) for sent in doc.sents], convert_to_tensor=True)

In [25]:
emb

tensor([[-1.3394e-01, -2.5056e-01, -1.0965e-01,  ..., -5.8495e-01,
         -1.1144e-01,  6.6581e-02],
        [-8.7038e-02,  2.1576e-01, -3.5745e-02,  ...,  2.0368e-01,
         -3.7795e-01, -7.4494e-02],
        [-8.9384e-02, -4.1365e-02, -3.2524e-04,  ...,  7.0949e-03,
         -1.1557e-01, -1.0126e-02],
        ...,
        [-3.4911e-01,  4.9186e-01,  8.5546e-02,  ..., -4.6969e-01,
         -1.1040e-01, -5.7458e-02],
        [-3.5540e-01,  6.0952e-02,  1.0769e-01,  ..., -2.0191e-01,
         -7.6770e-02, -1.4361e-03],
        [-1.6938e-01,  6.2705e-03,  2.6697e-02,  ..., -7.1487e-02,
         -3.4580e-01,  1.3832e-01]])

In [21]:
emb.shape

torch.Size([150, 768])

In [15]:
import torch
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to('cuda')

In [16]:
trg[:, :-1]

tensor([[1, 7, 4, 3, 5, 9, 2],
        [1, 5, 6, 2, 4, 7, 6]], device='cuda:0')